In [84]:
%matplotlib inline

"""
====================================================================
Create an RHI plot with reflectivity contour lines from an MDV file
====================================================================

An example which creates an RHI plot of velocity using a RadarDisplay object
and adding Reflectivity contours from the same MDV file.

"""
print __doc__

# Author: Cory Weber (cweber@anl.gov)
# License: BSD 3 clause
import matplotlib.pyplot as plt
import pyart
import numpy as np
import os
from astropy.convolution import convolve, convolve_fft
from astropy.convolution import Gaussian2DKernel

root_path = '/home/meso/boonah_srhi/odim_h5/'
sweep     = 0
nyq       = 50
v_min     = -12
v_max     = 12
r_min     = 0
r_max     = 70
# save figure
#plt.savefig(out_fn, dpi=100)



Create an RHI plot with reflectivity contour lines from an MDV file

An example which creates an RHI plot of velocity using a RadarDisplay object
and adding Reflectivity contours from the same MDV file.




In [85]:
def plot_rhi(in_fn,out_fn,elv_offset):
    #Since the names in the HDF5 file are non-standard we use file_field_names = True
    #Since the names in the HDF5 file are non-standard we use file_field_names = True
    radar = pyart.aux_io.read_odim_h5(in_fn, file_field_names=True) 
    
    #rename fields
    radar.fields['DBZH']['standard_name'] = 'Reflectivity'
    radar.fields['DBZH']['units'] = 'dBZ'
    radar.fields['DBZH']['long_name'] = 'Radar Reflectivity Factor'
    radar.fields['VRADH']['standard_name'] = 'Velocity'
    radar.fields['VRADH']['units'] = 'm/s'
    radar.fields['VRADH']['long_name'] = 'Radial Velocity of Scatterers'

    #offset elevation
    #radar.elevation['data'] = radar.elevation['data']+elv_offset

    #write and re-read
    #pyart.io.write_cfradial('temp.cf', radar)
    #radar = pyart.io.read_cfradial('temp.cf')               
    
    #exclude mask
    gatefilter = pyart.correct.GateFilter(radar)
    gatefilter.exclude_below('VRADH',v_min)
    gatefilter.exclude_above('VRADH',v_max)
    corr_vel = pyart.correct.dealias_region_based(
        radar, vel_field='VRADH', keep_original=False, 
        gatefilter = gatefilter, nyquist_vel=nyq, centered = True)
    radar.add_field('corrected_VRADH', corr_vel, replace_existing = True)
    
    
    display = pyart.graph.RadarDisplay(radar)
    
    #line commented out to show reflectivity
    fig = plt.figure(figsize=[10, 10])
    ax = fig.add_subplot(111)
    display.plot_rhi('DBZH', vmin=r_min, vmax=r_max, sweep=sweep, fig=fig)
    display.set_limits(xlim=[7, 27], ylim=[1, 15])
    
    # get data
    start = radar.get_start(sweep)
    end = radar.get_end(sweep) + 1
    vradh = radar.fields['VRADH']['data'][start:end]
    dbzh  = radar.fields['DBZH']['data'][start:end]
    vradh[vradh<v_min] = np.nan
    vradh[vradh>v_max] = np.nan    
    
    g = Gaussian2DKernel(stddev=3)
    vradh_interp = convolve(vradh, g,boundary='zero')
    vradh_interp[dbzh<0] = np.nan
    
    # calculate (R)ange 
    x = radar.gate_x['data'][start:end] / 1000.0
    y = radar.gate_y['data'][start:end] / 1000.0
    z = (radar.gate_z['data'][start:end] / 1000.0)
    R = np.sqrt(x ** 2 + y ** 2) * np.sign(y)
    
    # add contours
    # creates steps 35 to 100 by 5
    neg_levels = np.arange(-12,-2,2)
    pos_levels = np.arange(2,12,2)
    # adds coutours to plot\
    neg_contours = ax.contour(R, z, vradh_interp, neg_levels, linewidths=1.5, colors='k',
                          linestyles='dashed', antialiased=True)
    zero_contours = ax.contour(R, z, vradh_interp, 0, linewidths=2, colors='#6E6E6E',
                          linestyles='dashdot', antialiased=True)
    pos_contours = ax.contour(R, z, vradh_interp, pos_levels, linewidths=1.5, colors='k',
                          linestyles='solid', antialiased=True)

    # adds contour labels (fmt= '%r' displays 10.0 vs 10.0000)
    #plt.clabel(neg_contours, neg_levels, fmt='%r', inline=True, fontsize=10)
    #plt.clabel(pos_contours, pos_levels, fmt='%r', inline=True, fontsize=10)

    # format plot
    # add grid (dotted lines, major axis only)
    ax.grid(color='k', linestyle=':', linewidth=1, which='major')
    # save figure
    plt.savefig(out_fn, dpi=100)
    

In [86]:
fls    = os.listdir(root_path)
fls.sort()

i = 1
out_dir = 'tmp/boonah_20151104/'

for fl in fls:
    print('processing file ',i,' ',root_path+ fl)
    if (i % 2 == 0):
        plot_rhi(root_path+ fl,out_dir+fl+'.png',0.4)
    else:
        plot_rhi(root_path+ fl,out_dir+fl+'.png',0)
    plt.close()
    i = i +1

('processing file ', 1, ' ', '/home/meso/boonah_srhi/odim_h5/uq-xpol_rhi_20151104_084215.h5')


/home/meso/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:44: MaskedArrayFutureWarning: setting an item on a masked array which has a shared mask will not copy the mask and also change the original mask array in the future.
Check the NumPy 1.11 release notes for more information.
/home/meso/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:45: RuntimeWarning: invalid value encountered in greater


('processing file ', 2, ' ', '/home/meso/boonah_srhi/odim_h5/uq-xpol_rhi_20151104_084222.h5')
('processing file ', 3, ' ', '/home/meso/boonah_srhi/odim_h5/uq-xpol_rhi_20151104_084228.h5')
('processing file ', 4, ' ', '/home/meso/boonah_srhi/odim_h5/uq-xpol_rhi_20151104_084234.h5')
('processing file ', 5, ' ', '/home/meso/boonah_srhi/odim_h5/uq-xpol_rhi_20151104_084240.h5')
('processing file ', 6, ' ', '/home/meso/boonah_srhi/odim_h5/uq-xpol_rhi_20151104_084247.h5')
('processing file ', 7, ' ', '/home/meso/boonah_srhi/odim_h5/uq-xpol_rhi_20151104_084253.h5')
('processing file ', 8, ' ', '/home/meso/boonah_srhi/odim_h5/uq-xpol_rhi_20151104_084259.h5')
('processing file ', 9, ' ', '/home/meso/boonah_srhi/odim_h5/uq-xpol_rhi_20151104_084306.h5')
('processing file ', 10, ' ', '/home/meso/boonah_srhi/odim_h5/uq-xpol_rhi_20151104_084312.h5')
('processing file ', 11, ' ', '/home/meso/boonah_srhi/odim_h5/uq-xpol_rhi_20151104_084318.h5')
('processing file ', 12, ' ', '/home/meso/boonah_srhi/odim